In [ ]:
import bs4
import urllib
from urllib.parse import urljoin
import gzip
import json
from os import makedirs
from os.path import join, isdir, exists
import time
import numpy as np
from random import randint
from hashlib import sha1
import pandas as pd

AZLYRICS_URL = 'http://www.azlyrics.com/t/taylorswift.html'
CACHE_DIR = 'html/'
data = pd.DataFrame(columns = ['lyrics'])

urlobject = urllib.request.urlopen(AZLYRICS_URL)
time.sleep(randint(5, 30))
print('Read %s' % AZLYRICS_URL)
full_html = urlobject.read()
soup = bs4.BeautifulSoup(full_html, "html.parser")

lyrics_urls = []
for item in soup.find_all('a',href = True):
    if '/lyrics/' in item['href']:
        lyrics_urls.append(('https://www.azlyrics.com'+item['href'][2:]))

data = []
for link in lyrics_urls:
    try:          
        lyrics_urlobject = urllib.request.urlopen(link)
        full_html = lyrics_urlobject.read()
        lyrics_soup = bs4.BeautifulSoup(full_html,'html.parser')
        lyrics = str(lyrics_soup)
        # lyrics lies between up_partition and down_partition
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->'
        down_partition = '<!-- MxM banner -->'
        lyrics = lyrics.split(up_partition)[1]
        lyrics = lyrics.split(down_partition)[0]
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').replace('<br/>','').strip()
        data.append(lyrics)
        time.sleep(randint(5, 30))
        print(len(data))
    except:
        pass
    

In [ ]:
import pickle
with open("taylor_swift.txt", "wb") as f:
    pickle.dump(data, f)

In [2]:
import pickle
with open("taylor_swift.txt", "rb") as f:
    mylist = pickle.load(f)

In [3]:
from __future__ import print_function #Python 2/3 compatibility for print statements
import pandas
pandas.set_option('display.max_colwidth', 300) #widen pandas rows display
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import Dense
from keras.layers import GRU,LSTM
from keras.layers import Embedding     
from keras.models import Sequential
from keras.models import Model

c:\users\khatwd2\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
'''
data = pandas.read_csv('data/example_train_stories.csv', encoding='utf-8')[:100]
#data[:10]
'''

In [4]:
import copy
t = Tokenizer()
#docs = data['Story'].tolist()
docs = copy.deepcopy(mylist)
#max_length = max([len(sent.split()) for sent in docs])
max_length = 10
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
print(vocab_size)
encoded_docs = t.texts_to_sequences(docs)
sequences = list()
for doc in encoded_docs:
    for i in range(1, len(doc)):
        if i <= max_length:
            sequence = doc[:i+1]
        else:
            sequence = doc[i-max_length:i]
            
        sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
#max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
#print(t.word_counts)
#print(t.document_count)
#print(t.word_index)
#print(t.word_docs)
#print(encoded_docs)

3516
Total Sequences: 67912
Max Sequence Length: 754


In [5]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length-1))
model.add(LSTM(50))
#model.add(LSTM(500,return_sequences=True))
#model.add(LSTM(500))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

In [ ]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=10, verbose=2, batch_size = 10)
model.save_weights("taylor_swift_model.h5")
print("Saved model to disk")

In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [ ]:
loaded_model.load_weights("taylor_swift_model.h5")
print("Loaded model from disk")
generate_seq(model,t,max_length,'look what you made me do',10)